In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
metadata = pd.read_csv("../master_metadata.csv")

In [3]:
metadata.loc[metadata["Season"].str.contains("Summer"), "Season"] = "Summer"

In [4]:
metadata["Notes"] = metadata["Notes"].fillna("NA")
metadata["Site"] = metadata["Site"].fillna("NA")

In [5]:
metadata = metadata[~metadata["Notes"].str.contains("Missing eDNA")]

In [6]:
metadata = metadata[~metadata["Site"].str.contains("Blank")].reset_index(drop=True)

In [19]:
asv = pd.read_csv("../dada2_asv_table_with_taxonomy.csv")
asv["Overall_Best"] = asv["Best_Hit_Refseq"]
asv["Overall_Best_Percent_ID"] = asv["Percent_ID_Refseq"]
asv["Overall_Best_Taxonomic_Level"] = asv['Best_Hit_Taxonomic_level_Refseq']
asv.loc[asv["Percent_ID_Refseq"] < 98, "Overall_Best"] = asv["Best_Hit"]
asv.loc[asv["Percent_ID_Refseq"] < 98, "Overall_Best_Percent_ID"] = asv["Percent_ID"]
asv.loc[asv["Percent_ID_Refseq"] < 98, "Overall_Best_Taxonomic_Level"] = asv['Best_Hit_Taxonomic_level']
asv = asv[asv["Class"] == "Actinopteri"].reset_index(drop=True)
asv = asv[asv["CountsAcross"] > 10].reset_index(drop=True)
asv = asv[asv["Overall_Best"].notna()].reset_index(drop=True)
asv = asv[asv["Overall_Best_Taxonomic_Level"] == "Species"]

In [26]:
lakes = metadata[["Name", "Season"]][metadata["ALSC"] == 0][metadata["Name"].notna()].drop_duplicates()

/tmp/ipykernel_3914923/1469803059.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lakes = metadata[["Name", "Season"]][metadata["ALSC"] == 0][metadata["Name"].notna()].drop_duplicates()


In [11]:
metadata = metadata[metadata["EGCF-ID"].notna()].reset_index(drop=True)

In [28]:
springspecies

,Overall_Best,nonzero


In [40]:
for lake in ["Dart Lake"]:
    springids = metadata["EGCF-ID"][metadata["Name"] == lake][metadata["Season"] == "Spring"].tolist()
    fallids = metadata["EGCF-ID"][metadata["Name"] == lake][metadata["Season"] == "Early Fall"].tolist()
    springspecies = asv[springids + ["Overall_Best"]]
    fallspecies = asv[fallids + ["Overall_Best"]]
    springspecies["nonzero"] = springspecies[springids].astype(bool).sum(axis=1)
    fallspecies["nonzero"] = fallspecies[fallids].astype(bool).sum(axis=1)

/tmp/ipykernel_3914923/1566370548.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  springspecies["nonzero"] = springspecies[springids].astype(bool).sum(axis=1)
/tmp/ipykernel_3914923/1566370548.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fallspecies["nonzero"] = fallspecies[fallids].astype(bool).sum(axis=1)


In [42]:
for lake in lakes["Name"].unique():
    springids = metadata["EGCF-ID"][metadata["Name"] == lake][metadata["Season"] == "Spring"].tolist()
    fallids = metadata["EGCF-ID"][metadata["Name"] == lake][metadata["Season"] == "Early Fall"].tolist()
    springspecies = asv[springids + ["Overall_Best"]]
    fallspecies = asv[fallids + ["Overall_Best"]]
    springspecies["nonzero"] = springspecies[springids].astype(bool).sum(axis=1)
    fallspecies["nonzero"] = fallspecies[fallids].astype(bool).sum(axis=1)
    springspecies = springspecies[springspecies["nonzero"] > 0].reset_index(drop=True)
    fallspecies = fallspecies[fallspecies["nonzero"] > 0].reset_index(drop=True)
    springspp = springspecies["Overall_Best"].unique().tolist()
    fallspp = fallspecies["Overall_Best"].unique().tolist()
    print(lake, [i + " spring" for i in springspp if i not in fallspp] + [i + " fall" for i in fallspp if i not in springspp])

Dart Lake ['Ameiurus nebulosus spring', 'Chrosomus eos spring']
Sagamore ['Micropterus salmoides fall', 'Chrosomus neogaeus fall', 'Salmo salar fall', 'Coregonus clupeaformis fall', 'Lepomis macrochirus fall']
Moss Lake ['Rhinichthys atratulus spring', 'Semotilus atromaculatus fall', 'Sander vitreus fall']
Little Clear ['Perca flavescens spring', 'Micropterus salmoides spring', 'Mylopharyngodon piceus spring', 'Catostomus commersonii spring', 'Lepomis gibbosus fall']
Heart ['Lepomis gibbosus spring', 'Salvelinus fontinalis spring', 'Micropterus salmoides fall']
Rondaxe ['Umbra limi spring', 'Chrosomus neogaeus spring', 'Chrosomus eos spring', 'Pimephales notatus spring']
East Copperas ['Lepomis macrochirus spring', 'Fundulus diaphanus spring', 'Siniperca knerii x Siniperca chuatsi spring', 'Perca flavescens fall', 'Chrosomus neogaeus fall', 'Ameiurus melas fall', 'Lateolabrax maculatus fall']
Upper Cascade ['Hypophthalmichthys nobilis spring', 'Salmo trutta spring', 'Salvelinus conflue

/tmp/ipykernel_3914923/4288318997.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  springspecies["nonzero"] = springspecies[springids].astype(bool).sum(axis=1)
/tmp/ipykernel_3914923/4288318997.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fallspecies["nonzero"] = fallspecies[fallids].astype(bool).sum(axis=1)
/tmp/ipykernel_3914923/4288318997.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [169]:
fallids = metadata["EGCF-ID"][metadata["Name"] == "Sagamore"][metadata["Season"] == "Early Fall"].tolist()

In [170]:
springspecies = asvs[springids + ["species"]]

In [171]:
fallspecies = asvs[fallids + ["species"]]

In [172]:
springspecies["nonzero"] = springspecies[springids].astype(bool).sum(axis=1)
fallspecies["nonzero"] = fallspecies[fallids].astype(bool).sum(axis=1)
springspecies = springspecies[springspecies["nonzero"] > 0].reset_index(drop=True)
fallspecies = fallspecies[fallspecies["nonzero"] > 0].reset_index(drop=True)

/tmp/ipykernel_597029/2644553305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  springspecies["nonzero"] = springspecies[springids].astype(bool).sum(axis=1)
/tmp/ipykernel_597029/2644553305.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fallspecies["nonzero"] = fallspecies[fallids].astype(bool).sum(axis=1)


In [173]:
springspp = springspecies["species"].unique().tolist()
fallspp = fallspecies["species"].unique().tolist()

In [175]:
[i + " spring" for i in springspp if i not in fallspp] + [i + " fall" for i in fallspp if i not in springspp]

['Carassius spring',
 'Ancherythroculter spring',
 'Chrosomus neogaeus fall',
 'Coregonus fall',
 'Salmo salar fall',
 'Bos fall',
 'Eurycea bislineata fall',
 'Sus scrofa fall',
 'Coregonus clupeaformis fall',
 'Micropterus salmoides fall']